In [ ]:
# Not neccessary since these packages come with the runtime
#!pip install xgboost snowflake-ml-python 

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
import ast
#add another package
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
titanic = pd.read_csv('titanic_snowflake.csv')
titanic = titanic.drop(["AGE", 
                        "DECK", 
                        "ALIVE",
                        "ADULT_MALE",
                        "EMBARKED",
                        "PCLASS",
                        "ALONE",
                        "SEX"],axis=1)
titanic.head()

In [ ]:
# The next two cells are optional
# This is showing if your data is already in snowflake how to 
# turn it into a pandas dataframe


# This step turns pandas -> snowpark and writes to snowflake
titanic_sf = session.create_dataframe(titanic)
titanic_sf.write.mode("overwrite").save_as_table("titanic_raw")

In [ ]:
# Here we read a table from Snowflake into a Snowpark dataframe

titanic_raw = session.table('titanic_raw')
titanic_raw.show()

In [ ]:
Select * from titanic_raw;

In [ ]:
table  = 'titanic_raw'

df_demo = session.sql(
f'''
Select * from {table};
'''
).to_pandas()
df_demo

In [ ]:
# Here we read a table from Snowflake into a Pandas dataframe

titanic_raw = session.table('titanic_raw').to_pandas()
titanic_raw.head()

In [ ]:
titanic.dropna(inplace=True)

In [ ]:
titanic = pd.get_dummies(titanic, drop_first=True)

# Convert all boolean columns to integers
titanic = titanic.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

titanic.dtypes

In [ ]:
#now we will get the train data and label
x = titanic.drop('SURVIVED',axis=1)
y = titanic.SURVIVED

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=.85,random_state=1234)

In [ ]:
param_grid = {
    "n_estimators": [100, 200],
    "learning_rate": [0.1, 0.5],
    "max_depth": [1,2,3,4,5,6],
    "min_child_weight": [1, 6]
}

In [ ]:
model = XGBClassifier(objective='binary:logistic', 
                      eval_metric='logloss')

grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid)

grid_search.fit(xtrain, ytrain)

In [ ]:
# Best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(xtest, ytest)
print("Test Score:", test_score)

In [ ]:
metrics = {
    "Accuracy": best_score,
    "Params": best_params
}

metrics

In [ ]:
# import pickle

# MY_STAGE = 'ML_STAGE'
# MY_FILE_NAME = "model.pkl"
 
# pickle.dump(best_model, open('model.pkl','wb'))

# # Upload the file to a stage.
# put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
# put_result[0].status

In [ ]:
# from pickle import load

# model_pkl = session.file.get("@ML_STAGE/model.pkl","titanic_cr")

# #deserialize (unpickle) the model to use on a new data set
# titanic_pkl=load(open('model.pkl','rb'))

In [ ]:
from snowflake.ml.registry import Registry

# Get sample input data to pass into the registry logging function
X = xtrain.sample(n=1)

# Create a registry and log the model
# You can specify a different DB and Schema if you'd like
# otherwise it uses the session context
# If a registry does not exist it will create one
reg = Registry(session=session)

# Define model name and version (use uppercase for name)
model_name = "TITANIC"

titanic_model = reg.log_model(
    model_name=model_name,
    options = {
    "relax_version": True,
    },
    target_platforms=["WAREHOUSE", "SNOWPARK_CONTAINER_SERVICES"],
    #version_name="V_1", # If you leave version_name off SF creates one
    model=best_model,
    sample_input_data=X,
    metrics=metrics,
)

In [ ]:
from snowflake.ml.registry import Registry

# Get sample input data to pass into the registry logging function
X = xtrain.sample(n=1)

# Create a registry and log the model
# You can specify a different DB and Schema if you'd like
# otherwise it uses the session context
# If a registry does not exist it will create one
reg = Registry(session=session)

# Define model name and version (use uppercase for name)
model_name = "TITANIC"

titanic_model = reg.log_model(
    model_name=model_name,
    options = {
    "relax_version": True,
    },
    target_platforms=["WAREHOUSE", "SNOWPARK_CONTAINER_SERVICES"],
    #version_name="V_1", # If you leave version_name off SF creates one
    model=best_model,
    sample_input_data=X,
    metrics=metrics,
)

In [ ]:
models_df = reg.show_models()
models_df[models_df['name'] == model_name]

In [ ]:
models = reg.get_model(model_name).show_versions()
models.sort_values(by='created_on', ascending=False)

In [ ]:
recent_model_name = models.sort_values('created_on', ascending=False).iloc[0]['name']
print(recent_model_name)

In [ ]:
m = reg.get_model(model_name)
m.default = recent_model_name
mv = m.default
mv.version_name

In [ ]:
remote_prediction = mv.run(xtest, function_name="PREDICT_PROBA")
remote_prediction.head()

In [ ]:
test_sf = session.create_dataframe(xtest)
test_sf.write.mode("overwrite").save_as_table("test_pd")
session.table('test_pd').show()

In [ ]:
titanic_sf = session.create_dataframe(xtest)
titanic_sf.write.mode("overwrite").save_as_table("titanic_predict")

In [ ]:
select *, round(TITANIC!predict_proba(*):output_feature_0,2)
as surv_pred
from titanic_predict